<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/master/signate-23%E5%9B%9E%E7%9B%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [118]:
# LightGBM
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# Imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler


In [119]:
#control
!pip install git+https://github.com/pandas-profiling/pandas-profiling.git
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pandas_profiling
import seaborn as sns
from pandas_profiling.utils.cache import cache_file

#Optuna
!pip install optuna
import optuna
from sklearn.metrics import log_loss

#何で必要なのか分かってない
%matplotlib inline
warnings.filterwarnings('ignore')

  Cloning https://github.com/pandas-profiling/pandas-profiling.git to /tmp/pip-req-build-2jrywfoy
  Running command git clone -q https://github.com/pandas-profiling/pandas-profiling.git /tmp/pip-req-build-2jrywfoy
  Created wheel for pandas-profiling: filename=pandas_profiling-2.9.0rc1-py2.py3-none-any.whl size=258106 sha256=7894e0b671b90e2b014e2f198bdf8d12327516108dc4f2a6afb74b5e1d290c70
  Stored in directory: /tmp/pip-ephem-wheel-cache-bhtyh683/wheels/cd/13/75/8fcecd52c706914e90d916ede57f8c74de60e4c9ebc6c2f3b7
Successfully built pandas-profiling
  Using cached https://files.pythonhosted.org/packages/06/b0/9a6313c78bca92abfacc08a2ad8b27bfe845256f615786ee2b6452ae1978/optuna-2.0.0.tar.gz
     |████████████████████████████████| 1.1MB 2.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |█████████

In [120]:
#train,test,submit_sampleのみっつがそろっているか確認
!ls

sample_data		       submit_sample.csv  train.csv
submission_lightgbm_kfold.csv  test.csv


In [121]:
#そろっていなかったら選択
from google.colab import files
train_up = files.upload()

MessageError: ignored

In [252]:
sub = pd.read_csv("submit_sample.csv",names=("A","B"))
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")


In [253]:

data = pd.concat([train,test], sort=False)

#特徴量をエンジニアリング
data["income"]=data["job"].copy()
data["goodincome"]=data["job"].copy()
data["job"].replace(['blue-collar','management','technician','admin.','services',\
                     'unknown','self-employed','entrepreneur','student','retired',\
                     'unemployed','housemaid'],[0,1,2,3,4,5,6,7,8,9,10,11],inplace=True)
data["loan"].replace(['yes','no'],[1,0],inplace=True)
data["married"]=data["marital"]
data["single"]=data["marital"]
data['education'].replace(['secondary','tertiary','primary','unknown'],[3,1,0,2],inplace=True)
data["housing"].replace(['yes','no'],[1,0],inplace=True)
data["contact"].replace(['cellular','telephone','unknown'],[2,1,0],inplace=True)
data["poutcome"].replace(['success','failure','unknown','other'],[3,1,2,0],inplace=True)
#data["job"] = data["job"].astype(np.int64)

data["month"].replace(['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','des'],\
                      [3,4,5,6,7,8,9,10,11,0,1,2],inplace=True)
#data["y"].astype(np.int64)
#ここから追加
data["year"]=data["month"]*30+data["day"]
data["campaign+previous"]=data["campaign"] + data["previous"]
data["poutcome/pdays"]=data["poutcome"] / (data["pdays"]+10) * 10000
data["poutcome/pdays"] = data["poutcome/pdays"].astype(np.int64)
data["income"].replace(['blue-collar','management','technician',\
                                         'admin.','services','unknown','self-employed',\
                                         'entrepreneur','student','retired','unemployed','housemaid'],\
                                        [0,0,0,0,0,0,0,0,1,1,1,0],inplace=True)
data["goodincome"].replace(['blue-collar','management','technician','admin.','services','unknown','self-employed','entrepreneur','student','retired','unemployed','housemaid'],\
                                                [0,1,0,1,0,0,1,1,0,0,0,0],inplace=True)
data["housing+loan"] = data["housing"] + data["loan"]
data["duration*campaign"] = data["duration"] * data["campaign"] 
data["married"].replace(['married','single','divorced'],[1,0,1],inplace=True)
data["single"].replace(['married','single','divorced'],[1,0,0],inplace=True)
train["y"]=train["y"].astype(np.int64)

#全体のうち、残したいモノだけ選ぶ
all_columns = list(train.columns)
remain_columns = ['age','job','education','balance','housing','loan',
                  'contact','day','month','duration','campaign','pdays','previous',
                  'poutcome','y','income','goodincome',"married","single","year",
                  'campaign+previous'	,'poutcome/pdays'	,'housing+loan','duration*campaign']
                  
delete_columns = list(set(all_columns)-set(remain_columns))
data.drop(delete_columns, axis=1, inplace=True)

#trainとtestを再度切り分け
train = data[:len(train)]
test = data[len(train):]

#train,testを、さらに説明変数Xと、予測変数yに切り分け。y_testは与えられていないのでなし。三種類がでる
y = train['y']
X = train.drop('y', axis = 1)
X_sub = test.drop('y', axis = 1)
#Xcolumns=X_train.columns

In [254]:
len(y[y == 0]), len(y[y == 1])

(24988, 2112)

In [255]:
len(y[y == 1])/len(y)

0.07793357933579335

In [256]:
X

,age,job,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,income,goodincome,married,single,year,campaign+previous,poutcome/pdays,housing+loan,duration*campaign
0,31,4,3,12294,1,0,2,21,1,101,3,498,0,0,0,0,1,1,51,3,0,1,303
1,29,7,1,43027,0,0,2,22,10,158,2,702,0,2,0,1,0,0,322,2,28,0,316
2,35,1,1,12252,1,0,2,11,1,351,1,826,0,1,0,1,1,1,41,1,11,1,351
3,31,2,3,99121,1,1,0,16,7,658,2,120,0,1,0,0,1,1,226,2,76,2,1316
4,48,10,0,42005,1,0,1,3,6,177,1,273,0,2,1,0,1,1,183,1,70,1,177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27095,37,0,3,26661,1,0,2,27,7,345,4,425,0,2,0,0,1,1,237,4,45,1,1380
27096,35,4,3,42150,1,0,2,27,7,121,1,719,0,2,0,0,1,1,237,1,27,1,121
27097,35,4,2,34531,0,0,2,28,8,177,2,121,0,2,0,0,1,1,268,2,152,0,354
27098,30,3,3,99621,1,0,2,27,7,121,1,100,0,2,0,1,0,0,237,1,181,1,121


In [257]:
len(X)

27100

In [258]:
len(y)

27100

In [259]:
y

0        0.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
27095    0.0
27096    0.0
27097    0.0
27098    0.0
27099    0.0
Name: y, Length: 27100, dtype: float64

In [260]:
sum(y)

2112.0

In [261]:
def imbalanced_data_split(X, y, test_size=0.2):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        return X_train, X_test, y_train, y_test

In [262]:
X_train, X_test, y_train, y_test = imbalanced_data_split(X.values, y.values, test_size=0.2)
X_train2, X_valid, y_train2, y_valid = imbalanced_data_split(X_train, y_train, test_size=0.2)

In [263]:
X_train

array([[ 30,   7,   1, ...,  56,   0, 152],
       [ 60,   9,   3, ...,  83,   1, 303],
       [ 31,   2,   0, ...,  25,   1, 121],
       ...,
       [ 29,   7,   2, ...,  43,   0, 826],
       [ 30,   0,   3, ...,  27,   1, 363],
       [ 31,   0,   3, ...,  58,   1, 304]], dtype=int64)

In [264]:
len(X_train)

21680

In [265]:
len(y_train)

21680

In [266]:
y_test

array([0., 0., 0., ..., 0., 0., 0.])

In [267]:
len(y_test)

5420

# LightGBM

In [268]:
lgbm_params = {
    'learning_rate': 0.05,
    'num_leaves': 40,
    'boosting_type' : 'gbdt',
    'reg_alpha' : 1,
    'reg_lambda' : 1,
    'objective': 'binary',
    'max_bin': 300,
    'metric': 'auc',
}

#categorical_features = ['job','education','housing','loan','contact','poutcome',"income",'goodincome','married',"single",'housing+loan']
#X_train[categorical_features]=X_train[categorical_features].astype('category')

In [269]:
def lgbm_train(X_train_df, X_valid_df, y_train_df, y_valid_df, lgbm_params):
    lgb_train = lgb.Dataset(X_train_df, y_train_df)
    lgb_eval = lgb.Dataset(X_valid_df, y_valid_df, reference=lgb_train)

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train,
                      # モデルの評価用データを渡す
                      valid_sets=lgb_eval,
                      # 最大で 1000 ラウンドまで学習する
                      num_boost_round=1000,
                      # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                      early_stopping_rounds=10)
    
    return model

In [270]:
%%time
model_normal = lgbm_train(X_train2, X_valid, y_train2, y_valid, lgbm_params)

[1]	valid_0's auc: 0.805251
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.815825
[3]	valid_0's auc: 0.817747
[4]	valid_0's auc: 0.8218
[5]	valid_0's auc: 0.825149
[6]	valid_0's auc: 0.827336
[7]	valid_0's auc: 0.830136
[8]	valid_0's auc: 0.830165
[9]	valid_0's auc: 0.831059
[10]	valid_0's auc: 0.831259
[11]	valid_0's auc: 0.833037
[12]	valid_0's auc: 0.833967
[13]	valid_0's auc: 0.83466
[14]	valid_0's auc: 0.835408
[15]	valid_0's auc: 0.837015
[16]	valid_0's auc: 0.837062
[17]	valid_0's auc: 0.837236
[18]	valid_0's auc: 0.839483
[19]	valid_0's auc: 0.839535
[20]	valid_0's auc: 0.840581
[21]	valid_0's auc: 0.841271
[22]	valid_0's auc: 0.841602
[23]	valid_0's auc: 0.84293
[24]	valid_0's auc: 0.844172
[25]	valid_0's auc: 0.845816
[26]	valid_0's auc: 0.845874
[27]	valid_0's auc: 0.847107
[28]	valid_0's auc: 0.848084
[29]	valid_0's auc: 0.848422
[30]	valid_0's auc: 0.848841
[31]	valid_0's auc: 0.849253
[32]	valid_0's auc: 0.849786
[33]	valid_0's auc: 0.8

In [271]:
# テストデータを予測する
y_pred_normal = model_normal.predict(X_test, num_iteration=model_normal.best_iteration)

# auc を計算する
auc = roc_auc_score(y_test, y_pred_normal)
print(auc)

0.850349618520394


# Optuna


# Imbalanced-learn

In [272]:
sampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
X_train2, X_valid, y_train2, y_valid = imbalanced_data_split(X_resampled, y_resampled, test_size=0.2)

In [273]:
%%time
model_under_sample = lgbm_train(X_train2, X_valid, y_train2, y_valid, lgbm_params)

[1]	valid_0's auc: 0.782838
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.783844
[3]	valid_0's auc: 0.787057
[4]	valid_0's auc: 0.790619
[5]	valid_0's auc: 0.793744
[6]	valid_0's auc: 0.79669
[7]	valid_0's auc: 0.798335
[8]	valid_0's auc: 0.799784
[9]	valid_0's auc: 0.800313
[10]	valid_0's auc: 0.801469
[11]	valid_0's auc: 0.802383
[12]	valid_0's auc: 0.802952
[13]	valid_0's auc: 0.801346
[14]	valid_0's auc: 0.800939
[15]	valid_0's auc: 0.802423
[16]	valid_0's auc: 0.802283
[17]	valid_0's auc: 0.803136
[18]	valid_0's auc: 0.803788
[19]	valid_0's auc: 0.80385
[20]	valid_0's auc: 0.804099
[21]	valid_0's auc: 0.804742
[22]	valid_0's auc: 0.804598
[23]	valid_0's auc: 0.805136
[24]	valid_0's auc: 0.80581
[25]	valid_0's auc: 0.806038
[26]	valid_0's auc: 0.806388
[27]	valid_0's auc: 0.806169
[28]	valid_0's auc: 0.807373
[29]	valid_0's auc: 0.808428
[30]	valid_0's auc: 0.808664
[31]	valid_0's auc: 0.809369
[32]	valid_0's auc: 0.809193
[33]	valid_0's auc: 0.

In [274]:
# テストデータを予測する
y_pred_under_sample = model_under_sample.predict(X_test, num_iteration=model_under_sample.best_iteration)

# auc を計算する
auc = roc_auc_score(y_test, y_pred_under_sample)
print(auc)

0.8430898425721


# Bagging

In [275]:
def bagging(seed):
    sampler = RandomUnderSampler(random_state=seed, replacement=True)
    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    X_train2, X_valid, y_train2, y_valid = imbalanced_data_split(X_resampled, y_resampled, test_size=0.2)
    model_bagging = lgbm_train(X_train2, X_valid, y_train2, y_valid, lgbm_params)
    return model_bagging

In [276]:
%%time
models = []

for i in range(10):
    models.append(bagging(i))

[1]	valid_0's auc: 0.800335
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.798519
[3]	valid_0's auc: 0.799272
[4]	valid_0's auc: 0.798983
[5]	valid_0's auc: 0.798217
[6]	valid_0's auc: 0.805683
[7]	valid_0's auc: 0.805727
[8]	valid_0's auc: 0.810156
[9]	valid_0's auc: 0.812143
[10]	valid_0's auc: 0.811465
[11]	valid_0's auc: 0.812218
[12]	valid_0's auc: 0.812668
[13]	valid_0's auc: 0.812787
[14]	valid_0's auc: 0.812524
[15]	valid_0's auc: 0.812546
[16]	valid_0's auc: 0.814533
[17]	valid_0's auc: 0.815767
[18]	valid_0's auc: 0.816783
[19]	valid_0's auc: 0.817684
[20]	valid_0's auc: 0.816577
[21]	valid_0's auc: 0.817124
[22]	valid_0's auc: 0.817216
[23]	valid_0's auc: 0.817531
[24]	valid_0's auc: 0.818113
[25]	valid_0's auc: 0.818376
[26]	valid_0's auc: 0.818726
[27]	valid_0's auc: 0.819443
[28]	valid_0's auc: 0.81912
[29]	valid_0's auc: 0.819146
[30]	valid_0's auc: 0.819452
[31]	valid_0's auc: 0.820061
[32]	valid_0's auc: 0.819842
[33]	valid_0's auc: 

In [277]:
y_preds = []

for m in models:
    y_preds.append(m.predict(X_test, num_iteration=m.best_iteration))

y_preds_bagging = sum(y_preds)/len(y_preds)
# auc を計算する
auc = roc_auc_score(y_test, y_preds_bagging)
print(auc)

0.8476618135405821


In [278]:
y_test

array([0., 0., 0., ..., 0., 0., 0.])

In [279]:
sum(y_test)

422.0

In [280]:
len(y_test)

5420

In [281]:
models

In [282]:
y_preds_bagging

array([0.40682771, 0.47081367, 0.63580271, ..., 0.27380271, 0.31215199,
       0.47963997])

In [283]:
len(y_preds_bagging)

5420

In [284]:
y_test

array([0., 0., 0., ..., 0., 0., 0.])

In [285]:
len(y_test)

5420

In [286]:
for y_pred in y_preds:
    print(roc_auc_score(y_test, y_pred))

0.8351283641418653
0.8422176927643095
0.8341926818120613
0.8220371086823356
0.8392314271680235
0.8371068806669588
0.837431181003207
0.8379342258230306
0.8193692168810652
0.8358305407471045


In [287]:
y_subs = []

for m in models:
    y_subs.append(m.predict(X_sub, num_iteration=m.best_iteration))

y_subs_bagging = sum(y_subs)/len(y_subs)
# auc を計算する
y_subs_bagging

array([0.93260942, 0.28383637, 0.20715047, ..., 0.50886412, 0.07996322,
       0.6520975 ])

In [290]:
percentage=0.32
y_sub = (y_subs_bagging > percentage).astype(int)

sub['B'] = y_sub
sub.to_csv('submission_lightgbm_kfold.csv', index=False, header=False)

sum(sub["B"]),len(sub["B"])

(9611, 18050)

In [291]:
sum(y_sub)

9611

In [292]:
files.download('submission_lightgbm_kfold.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [293]:
def objective(trial):
    params = {
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': 0.05,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid, y_pred_valid)
    return score

In [294]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.217688	valid_1's binary_logloss: 1.00872
[20]	training's binary_logloss: 0.195256	valid_1's binary_logloss: 0.895166
[30]	training's binary_logloss: 0.180891	valid_1's binary_logloss: 0.822762
[40]	training's binary_logloss: 0.169985	valid_1's binary_logloss: 0.762963
[50]	training's binary_logloss: 0.15986	valid_1's binary_logloss: 0.708496
[60]	training's binary_logloss: 0.151197	valid_1's binary_logloss: 0.666988
[70]	training's binary_logloss: 0.143329	valid_1's binary_logloss: 0.626358
[80]	training's binary_logloss: 0.136186	valid_1's binary_logloss: 0.590633
[90]	training's binary_logloss: 0.129867	valid_1's binary_logloss: 0.560449
[100]	training's binary_logloss: 0.124222	valid_1's binary_logloss: 0.532001
[110]	training's binary_logloss: 0.118826	valid_1's binary_logloss: 0.502306
[120]	training's binary_logloss: 0.114102	valid_1's binary_logloss: 0.4803
[130]	training's binary_log

[I 2020-08-23 03:44:17,172] Trial 0 finished with value: 0.026781051256995352 and parameters: {'max_bin': 427, 'num_leaves': 79}. Best is trial 0 with value: 0.026781051256995352.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.215271	valid_1's binary_logloss: 0.991995
[20]	training's binary_logloss: 0.191506	valid_1's binary_logloss: 0.871958
[30]	training's binary_logloss: 0.175745	valid_1's binary_logloss: 0.795478
[40]	training's binary_logloss: 0.163522	valid_1's binary_logloss: 0.731207
[50]	training's binary_logloss: 0.152547	valid_1's binary_logloss: 0.671328
[60]	training's binary_logloss: 0.142932	valid_1's binary_logloss: 0.623255
[70]	training's binary_logloss: 0.134295	valid_1's binary_logloss: 0.578039
[80]	training's binary_logloss: 0.126637	valid_1's binary_logloss: 0.539438
[90]	training's binary_logloss: 0.120034	valid_1's binary_logloss: 0.509078
[100]	training's binary_logloss: 0.114275	valid_1's binary_logloss: 0.480163
[110]	training's binary_logloss: 0.109095	valid_1's binary_logloss: 0.455164
[120]	training's binary_logloss: 0.104391	valid_1's binary_logloss: 0.43097
[130]	training's binary_

[I 2020-08-23 03:44:27,085] Trial 1 finished with value: 0.012920924217732561 and parameters: {'max_bin': 372, 'num_leaves': 96}. Best is trial 1 with value: 0.012920924217732561.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.214776	valid_1's binary_logloss: 0.989285
[20]	training's binary_logloss: 0.190662	valid_1's binary_logloss: 0.865635
[30]	training's binary_logloss: 0.174693	valid_1's binary_logloss: 0.785841
[40]	training's binary_logloss: 0.162235	valid_1's binary_logloss: 0.720306
[50]	training's binary_logloss: 0.15082	valid_1's binary_logloss: 0.658601
[60]	training's binary_logloss: 0.14071	valid_1's binary_logloss: 0.604612
[70]	training's binary_logloss: 0.132381	valid_1's binary_logloss: 0.565581
[80]	training's binary_logloss: 0.124551	valid_1's binary_logloss: 0.523822
[90]	training's binary_logloss: 0.117553	valid_1's binary_logloss: 0.487456
[100]	training's binary_logloss: 0.111832	valid_1's binary_logloss: 0.46197
[110]	training's binary_logloss: 0.106748	valid_1's binary_logloss: 0.438357
[120]	training's binary_logloss: 0.101631	valid_1's binary_logloss: 0.415059
[130]	training's binary_lo

...

[I 2020-08-23 03:49:39,964] Trial 37 finished with value: 0.0498756413398959 and parameters: {'max_bin': 441, 'num_leaves': 63}. Best is trial 19 with value: 0.0046284648643910705.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.227009	valid_1's binary_logloss: 1.05919
[20]	training's binary_logloss: 0.210592	valid_1's binary_logloss: 0.979303
[30]	training's binary_logloss: 0.200549	valid_1's binary_logloss: 0.928695
[40]	training's binary_logloss: 0.192766	valid_1's binary_logloss: 0.891472
[50]	training's binary_logloss: 0.186653	valid_1's binary_logloss: 0.85924
[60]	training's binary_logloss: 0.181143	valid_1's binary_logloss: 0.830643
[70]	training's binary_logloss: 0.176259	valid_1's binary_logloss: 0.80139
[80]	training's binary_logloss: 0.17193	valid_1's binary_logloss: 0.776877
[90]	training's binary_logloss: 0.168017	valid_1's binary_logloss: 0.757859
[100]	training's binary_logloss: 0.16438	valid_1's binary_logloss: 0.739177
[110]	training's binary_logloss: 0.161085	valid_1's binary_logloss: 0.724339
[120]	training's binary_logloss: 0.157999	valid_1's binary_logloss: 0.708776
[130]	training's binary_logl

[I 2020-08-23 03:49:46,206] Trial 38 finished with value: 0.17219625828873986 and parameters: {'max_bin': 375, 'num_leaves': 33}. Best is trial 19 with value: 0.0046284648643910705.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.218569	valid_1's binary_logloss: 1.0089
[20]	training's binary_logloss: 0.197236	valid_1's binary_logloss: 0.903054
[30]	training's binary_logloss: 0.183346	valid_1's binary_logloss: 0.837906
[40]	training's binary_logloss: 0.17251	valid_1's binary_logloss: 0.779962
[50]	training's binary_logloss: 0.163131	valid_1's binary_logloss: 0.729505
[60]	training's binary_logloss: 0.15452	valid_1's binary_logloss: 0.685115
[70]	training's binary_logloss: 0.146852	valid_1's binary_logloss: 0.646805
[80]	training's binary_logloss: 0.140108	valid_1's binary_logloss: 0.60922
[90]	training's binary_logloss: 0.13432	valid_1's binary_logloss: 0.5793
[100]	training's binary_logloss: 0.129148	valid_1's binary_logloss: 0.555319
[110]	training's binary_logloss: 0.124446	valid_1's binary_logloss: 0.530334
[120]	training's binary_logloss: 0.119988	valid_1's binary_logloss: 0.507642
[130]	training's binary_logloss

[I 2020-08-23 03:49:54,386] Trial 39 finished with value: 0.034415119935764175 and parameters: {'max_bin': 320, 'num_leaves': 73}. Best is trial 19 with value: 0.0046284648643910705.


In [295]:
study.best_params

{'max_bin': 354, 'num_leaves': 120}

In [296]:
lgbm_params = {
    'learning_rate': 0.05,
    'num_leaves': 120,
    'boosting_type' : 'gbdt',
    'reg_alpha' : 1,
    'reg_lambda' : 1,
    'objective': 'binary',
    'max_bin': 354,
    'metric': 'auc',
}

#categorical_features = ['job','education','housing','loan','contact','poutcome',"income",'goodincome','married',"single",'housing+loan']
#X_train[categorical_features]=X_train[categorical_features].astype('category')

In [297]:
%%time
models = []

for i in range(10):
    models.append(bagging(i))

[1]	valid_0's auc: 0.797342
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.796309
[3]	valid_0's auc: 0.795805
[4]	valid_0's auc: 0.795709
[5]	valid_0's auc: 0.795937
[6]	valid_0's auc: 0.794926
[7]	valid_0's auc: 0.802274
[8]	valid_0's auc: 0.807351
[9]	valid_0's auc: 0.806143
[10]	valid_0's auc: 0.807557
[11]	valid_0's auc: 0.808668
[12]	valid_0's auc: 0.808366
[13]	valid_0's auc: 0.809723
[14]	valid_0's auc: 0.810261
[15]	valid_0's auc: 0.809732
[16]	valid_0's auc: 0.809955
[17]	valid_0's auc: 0.808878
[18]	valid_0's auc: 0.810406
[19]	valid_0's auc: 0.811062
[20]	valid_0's auc: 0.811242
[21]	valid_0's auc: 0.811531
[22]	valid_0's auc: 0.811404
[23]	valid_0's auc: 0.811351
[24]	valid_0's auc: 0.811045
[25]	valid_0's auc: 0.811876
[26]	valid_0's auc: 0.813329
[27]	valid_0's auc: 0.813631
[28]	valid_0's auc: 0.814629
[29]	valid_0's auc: 0.815338
[30]	valid_0's auc: 0.81529
[31]	valid_0's auc: 0.815058
[32]	valid_0's auc: 0.815089
[33]	valid_0's auc: 

In [298]:
y_preds = []

for m in models:
    y_preds.append(m.predict(X_test, num_iteration=m.best_iteration))

y_preds_bagging = sum(y_preds)/len(y_preds)
# auc を計算する
auc = roc_auc_score(y_test, y_preds_bagging)
print(auc)

0.8431230312030025


In [299]:
y_subs = []

for m in models:
    y_subs.append(m.predict(X_sub, num_iteration=m.best_iteration))

y_subs_bagging = sum(y_subs)/len(y_subs)
# auc を計算する
y_subs_bagging

array([0.89931745, 0.26417224, 0.21214072, ..., 0.5327362 , 0.10564403,
       0.63575175])

In [305]:
percentage=0.37
y_sub = (y_subs_bagging > percentage).astype(int)

sub['B'] = y_sub
sub.to_csv('submission_lightgbm_kfold.csv', index=False, header=False)

sum(sub["B"]),len(sub["B"])

(8831, 18050)

In [307]:
files.download('submission_lightgbm_kfold.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>